In [4]:
#print final board
def print_matrix(Title, M):
    print(Title)
    for row in M:
        print([round(x,3)+0 for x in row])

#print intermediate board
def print_matrices(Title1, M1, Title2, M2):
    print(Title1, '\t'*int(len(M1)/2)+"\t"*len(M1), Title2)
    for i in range(len(M1)):
        row1 = ['{0:+7.3f}'.format(x) for x in M1[i]]
        row2 = ['{0:+7.3f}'.format(x) for x in M2[i]]
        print(row1,'\t', row2)

#return a board of 0s to fill in with results of computations
def zeros_matrix(rows, cols):
    A = []
    for i in range(rows):
        A.append([])
        for j in range(cols):
            A[-1].append(0.0)
    return A

#make a copy of the board for computations
def copy_matrix(M):
    rows = len(M)
    cols = len(M[0])

    MC = zeros_matrix(rows, cols)

    for i in range(rows):
        for j in range(cols):
            MC[i][j] = M[i][j]

    return MC

#multiply given matrices A and B
def matrix_multiply(A,B):
    rowsA = len(A)
    colsA = len(A[0])
    rowsB = len(B)
    colsB = len(B[0])

    C = zeros_matrix(rowsA, colsB)

    for i in range(rowsA):
        for j in range(colsB):
            total = 0
            for ii in range(colsA):
                total += A[i][ii] * B[ii][j]
            C[i][j] = total

    return C

In [5]:
# A = [[5.,4.,3.,2.,1.],[4.,3.,2.,1.,5.],[3.,2.,9.,5.,4.],[2.,1.,5.,4.,3.],[1.,2.,3.,4.,5.]]
# B = [[48],[38],[64],[43],[42]]
A = [[5.,3.,1.],[3.,9.,4.],[1.,3.,5.]]
B = [[9.0],[16.0],[9.0]]

In [6]:
AM = copy_matrix(A)
n = len(A)
BM = copy_matrix(B)

print()

#create a list that allows for flexible number of indices
indices = list(range(n))
for fd in range(n): # fd = focus diagonal
    fdScaler = 1.0 / AM[fd][fd]
    #FIRST STEP: scale fd row with fd inverse, for every column 
    for j in range(n):
        AM[fd][j] *= fdScaler
    BM[fd][0] *= fdScaler
    
    # Print out current matrix after computation
    print_matrices('AM Matrix', AM, 'BM Matrix', BM)
    print()
    
    # SECOND STEP: iterate through and scale all rows except the fd row
    for i in indices[0:fd] + indices[fd+1:]:
        currRowScaler = AM[i][fd]
        #compute current row - current row scaler*fdRow for every column
        for j in range(n):
            AM[i][j] = AM[i][j] - currRowScaler * AM[fd][j]
        BM[i][0] = BM[i][0] - currRowScaler * BM[fd][0]
        
        # Print out current matrix after computation
        print_matrices('AM Matrix', AM, 
                               'BM Matrix', BM)
        print()


AM Matrix 				 BM Matrix
[' +1.000', ' +0.600', ' +0.200'] 	 [' +1.800']
[' +3.000', ' +9.000', ' +4.000'] 	 ['+16.000']
[' +1.000', ' +3.000', ' +5.000'] 	 [' +9.000']

AM Matrix 				 BM Matrix
[' +1.000', ' +0.600', ' +0.200'] 	 [' +1.800']
[' +0.000', ' +7.200', ' +3.400'] 	 ['+10.600']
[' +1.000', ' +3.000', ' +5.000'] 	 [' +9.000']

AM Matrix 				 BM Matrix
[' +1.000', ' +0.600', ' +0.200'] 	 [' +1.800']
[' +0.000', ' +7.200', ' +3.400'] 	 ['+10.600']
[' +0.000', ' +2.400', ' +4.800'] 	 [' +7.200']

AM Matrix 				 BM Matrix
[' +1.000', ' +0.600', ' +0.200'] 	 [' +1.800']
[' +0.000', ' +1.000', ' +0.472'] 	 [' +1.472']
[' +0.000', ' +2.400', ' +4.800'] 	 [' +7.200']

AM Matrix 				 BM Matrix
[' +1.000', ' +0.000', ' -0.083'] 	 [' +0.917']
[' +0.000', ' +1.000', ' +0.472'] 	 [' +1.472']
[' +0.000', ' +2.400', ' +4.800'] 	 [' +7.200']

AM Matrix 				 BM Matrix
[' +1.000', ' +0.000', ' -0.083'] 	 [' +0.917']
[' +0.000', ' +1.000', ' +0.472'] 	 [' +1.472']
[' +0.000', ' +0.000', ' +3.

In [7]:
print("Now we multiply the original A matrix times our solution for X, which is BM.")
print_matrix('If we get our starting X, our solution is proved.\n', matrix_multiply(A,BM))

Now we multiply the original A matrix times our solution for X, which is BM.
If we get our starting X, our solution is proved.

[9.0]
[16.0]
[9.0]
